In [1]:
import numpy as np
import networkx as nx
import scipy
from karateclub.graph_embedding import FeatherGraph
import torch
from qtensor import QAOA_energy

/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-25 12:08:26,991- WARNING•	QuickBB solver is unavailable
2023-10-25 12:08:26,992- WARNING•	Tamaki solver is unavailable: No path /home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/thirdparty/tamaki_treewidth
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refa

In [2]:
def matrices_to_graphs(matrix_list):
    '''
    This function takes a list of adjacency matrices and generates a list of graphs.
    '''

    g_list = []
    for matrix in matrix_list:
        array = np.array(matrix)
        g = nx.from_numpy_array(array)
        g_list.append(g)

    return g_list

In [3]:
training_file = open("1045_40_node_random_graphs.txt")
training_matrix_list = np.loadtxt(training_file).reshape(1045, 40, 40)
training_graph_list = matrices_to_graphs(training_matrix_list)

In [4]:
test_file = open("graphs/graphs/204_100_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(204, 100, 100)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [5]:
model = FeatherGraph()
model.fit(training_graph_list)
model_array = model.get_embedding()

/home/jfalla/anaconda3/envs/qaoa/lib/python3.8/site-packages/karateclub/graph_embedding/feathergraph.py:110: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph, nodelist=range(self.n_nodes))


In [58]:
def find_indices(vector):
    '''
    This function returns the indices from the first three closest and last three furthest graphs
    in embedded space to the test graph. The first index in the array corresponds to the graph that
    is most similar, while the last index corresponds to the most dissimilar.
    '''
    
    length = len(vector)
    sorted_vector = sorted(vector)
    
    max_value = sorted_vector[length - 1]
    second_max_value = sorted_vector[length - 2]
    third_max_value = sorted_vector[length - 3]
    
    min_value = sorted_vector[0]
    second_min_value = sorted_vector[1]
    third_min_value = sorted_vector[2]
    
    for i in range(len(vector)):
        if vector[i] == max_value:
            max_index = vector.index(vector[i])
        if vector[i] == second_max_value:
            second_max_index = vector.index(vector[i])
        if vector[i] == third_max_value:
            third_max_index = vector.index(vector[i])
        if vector[i] == min_value:
            min_index = vector.index(vector[i])
        if vector[i] == second_min_value:
            second_min_index = vector.index(vector[i])
        if vector[i] == third_min_value:
            third_min_index = vector.index(vector[i])
        else:
            continue
            
    indices = [min_index, second_min_index, third_min_index, third_max_index, second_max_index, max_index]
    
    return indices

In [43]:
def euclidean_distance(model_vector, infer_vector):
    
    diffs = []
    
    for i in range(len(model_vector)):
        
        diff = (model_vector[i] - infer_vector[0][i])**2
        diffs.append(diff)
    
    return np.sqrt(sum(diffs))

In [59]:
def minkowski_distance(model_vector, infer_vector, p):
    
    diffs = []
    
    for i in range(len(model_vector)):
        
        diff = (model_vector[i] - infer_vector[0][i])**p
        diffs.append(diff)
        
    mink_dist = (sum(diffs))**(1/p)
    
    return mink_dist

In [60]:
# For Minkowski distances
indices = []

for i in range(len(test_graph_list)):
    infer_vector = model.infer([test_graph_list[i]])
    minkowski_distances = []
    for j in range(len(model_array)):
        dist = minkowski_distance(model_array[j], infer_vector, 3)
        minkowski_distances.append(dist)
    index = find_indices(minkowski_distances)
    indices.append(index)

<ipython-input-59-217b335e444c>:10: RuntimeWarning: invalid value encountered in double_scalars
  mink_dist = (sum(diffs))**(1/p)


UnboundLocalError: local variable 'third_max_index' referenced before assignment

In [90]:
# For Euclidean distances
indices = []

for i in range(len(test_graph_list)):
    infer_vector = model.infer([test_graph_list[i]])
    euclidean_distances = []
    for j in range(len(model_array)):
        dist = euclidean_distance(model_array[j], infer_vector)
        euclidean_distances.append(dist)
    index = find_indices(euclidean_distances)
    indices.append(index)

In [31]:
# For pairwise distances
indices = []

for i in range(len(test_graph_list)):
    infer_vector = model.infer([test_graph_list[i]])
    pairwise_distances = []
    for j in range(len(model_array)):
        dist = np.linalg.norm(model_array[j] - infer_vector)
        pairwise_distances.append(dist)
    index = find_indices(pairwise_distances)
    indices.append(index)

In [91]:
test_file = open("graphs/graphs/204_100_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(204, 100, 100)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [92]:
new_indices = [indices[0], indices[101], indices[203]]

In [93]:
graph_list = [test_graph_list[0], test_graph_list[101], test_graph_list[203]]

In [10]:
s = 0

for i in range(len(test_graph_list)):
    if indices[i][0] == i:
        s += 1
        
s

1045

In [46]:
gamma_params_file = open('graph_data/training_set_optimal_gammas.txt')
gamma_params = np.loadtxt(gamma_params_file).reshape(1045, 20, 3)

beta_params_file = open('graph_data/training_set_optimal_betas.txt')
beta_params = np.loadtxt(beta_params_file).reshape(1045, 20, 3)

In [26]:
test_file = open("graphs/graphs/20_40_node_connected_ws_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(20, 40, 40)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [ ]:
index = 6 # 3 top indices 3 bottom indices
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(len(test_graph_list))]

for i in range(len(test_graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(test_graph_list[i], gamma=gamma_params[indices[i][j]][k], beta=beta_params[indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy

Edge iteration:  95%|█████████▌| 57/60 [00:01<00:00, 33.90it/s, Treewidth=12]

In [94]:
index = 6 # 3 top indices 3 bottom indices
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(len(graph_list))]

for i in range(len(graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(graph_list[i], gamma=gamma_params[new_indices[i][j]][k], beta=beta_params[new_indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy

Edge iteration: 100%|██████████| 150/150 [00:09<00:00, 15.25it/s, Treewidth=12]


In [95]:
average_transfer_energies

[[89.84287881603146,
  89.82677432689317,
  89.83508954245296,
  53.60971281201746,
  59.12472867764054,
  59.044849981484845],
 [97.57953762362709,
  95.01992388604914,
  96.78038233400092,
  78.82818537384347,
  91.38309474954474,
  80.42971431660334],
 [112.07831215972496,
  113.85895225811412,
  113.67619105110552,
  66.57470389495681,
  73.30908371100995,
  87.25113489915657]]